In [ ]:
import requests
#下載csv檔
csv_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=CSV'
res = requests.get(url=csv_url)
if res.ok:
    print("下載成功")
else:
    print("下載失敗")

In [ ]:
import io
import csv
csv_str:str = res.text
csv_file = io.StringIO(csv_str)
dict_reader = csv.DictReader(csv_file)
csv_data = list(dict_reader)

In [ ]:
from pydantic import BaseModel,Field,field_serializer,RootModel
from datetime import datetime

class Site(BaseModel):
    站點名稱:str = Field(alias='sitename')
    城鎮:str = Field(alias='county')
    aqi:int
    狀態:str = Field(alias='status')
    日期:datetime = Field(alias='datacreationdate')

    @field_serializer("日期")
    def serialize_str(self,value):
        return value.strftime('%a %d %b %Y, %I:%M%p')

class Csvroot(RootModel):
    root:list[Site]

    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]

In [ ]:
csv_root = Csvroot.model_validate(csv_data)
json_str = csv_root.model_dump_json()

In [ ]:
with open('新aqi.json',mode='w',encoding='utf8') as jsonFile:
    jsonFile.write(json_str)